In [4]:
## Generate a graph
using StatsBase
using Combinatorics

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/hypergraph_louvain.jl")
include("jl/inference.jl")

estimateΩEmpirically (generic function with 1 method)

In [18]:
n = 200
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = buildΩ(ω, α0, kmax)
## Sample
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((200,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  191, 192, 193, 194, 195, 196, 197, 198, 199, 200]
  E: Dict{Int64,Dict}
  D: Array{Int64}((200,)) [8, 7, 9, 2, 7, 8, 3, 5, 10, 10  …  5, 6, 10, 7, 8, 6, 9, 4, 14, 4]


In [10]:
## Run Louvains, Naive, and Faster version

@time Zlou = Naive_HyperLouvain(H,Ω;α=α0)
@time Zlou2 = HyperLouvain(H,kmax,Ω;α=α0)

mlou = modularity(H,Zlou,Ω,α=α0)
mlou2 = modularity(H,Zlou2,Ω,α=α0)

# This may not be exactly the same, but that's okay.
# Even a difference of 1e-14 in modularity change may lead you to make a 
# different greedy decision, but it's still a valid move.
@show mlou, mlou2
@show abs(mlou- mlou2)


Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
  1.446576 seconds (7.38 M allocations: 1.404 GiB, 21.24% gc time)

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
  0.055149 seconds (397.57 k allocations: 23.057 MiB, 17.93% gc time)
(mlou, mlou2) = (-1044.8812939800981894541501431190877458021759593975730240345001220703125, -1044.8821879378618530027187532617516385613498641760088503360748291015625)
abs(mlou - mlou2) = 0.00089395776366354856861014266389275917390477843582630157470703125


0.00089395776366354856861014266389275917390477843582630157470703125

In [14]:
## Try supernode Louvain
# Sanity check: having a singletons initialization is equivalent to naive louvain
Z = collect(1:n)
@time Znew, changed = SuperNodeStep(H,Z,kmax,Ω;α=α0)
mlou3 =  modularity(H,Znew,Ω,α=α0)


Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
  0.180047 seconds (636.46 k allocations: 43.588 MiB, 50.59% gc time)


-1044.8812939800981894541501431190877458021759593975730240345001220703125

In [17]:
## Now repeatedly call the SuperNodeStep until no more improvement is possible
@time Z = SuperNodeLouvain(H,kmax,Ω;α=α0)
msuper = modularity(H,Z,Ω,α=α0)

SuperNode Louvain: Phase 1

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
SuperNode Louvain: Phase 2

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
SuperNode Louvain: Phase 3

Louvain Iteration 1
  0.148006 seconds (809.73 k allocations: 53.141 MiB, 17.69% gc time)


-1043.8259629293173430704738864106817519683545469888485968112945556640625